# Breaching privacy

This notebook does the same job as the cmd-line tool `breach.py`, but also directly visualizes the user data and reconstruction

In [ ]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

In [ ]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=['attack=modern', 
                                                      'case=3_small_batch_imagenet'])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
    print('Attack settings are:')
    print(OmegaConf.to_yaml(cfg.attack))
          
device = torch.device(f'cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [ ]:
cfg.case.user.data_idx = 200
cfg.case.model='resnet18'

cfg.case.user.num_data_points = 8

cfg.case.user.local_diff_privacy.gradient_noise = 0.0
cfg.case.user.local_diff_privacy.per_example_clipping = 0.0

cfg.case.server.has_external_data = False

cfg.attack.restarts.num_trials = 1
cfg.attack.regularization.deep_inversion.scale = 0.0001
cfg.attack.regularization.total_variation.scale = 0.2
cfg.attack.optim.langevin_noise = 0.0

### Instantiate all parties

In [ ]:
user, server = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)

In [ ]:
print(user)
print(server)
print(attacker)

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [ ]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

In [ ]:
user.plot(true_user_data)

# Reconstruct user data

In [ ]:
reconstructed_user_data, stats = attacker.reconstruct(server_payload, shared_data, 
                                                      server.secrets, dryrun=cfg.dryrun)

In [ ]:
user.plot(reconstructed_user_data, scale=True)

In [ ]:
# How good is the reconstruction?
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, 
                                    server_payload, server.model, user.dataloader, setup=setup,
                                    order_batch=True, compute_full_iip=False, skip_rpsnr=True)

In [ ]:
user.plot(reconstructed_user_data, scale=False)